In [1]:
import os
from dotenv import load_dotenv
load_dotenv()
from typing import TypeVar
T = TypeVar('T')
from kv.api import KV
from kv.sql import SQLKV
from kv.azure.blob import BlobKV, BlobContainerKV

In [2]:
CONN_STR = os.environ['BLOB_CONN_STR']
kv = BlobKV.from_conn_str(CONN_STR)

In [ ]:
def parse(type: type[T], conn_str: str) -> KV[T]:
    if conn_str.startswith("azure+blob://"):
        from kv.azure.blob import BlobKV
        return BlobKV.validated(type, conn_str.split('://')[1])
    if conn_str.startswith("azure+blob+container://"):
        parts = conn_str.split('://')[1].split(';', maxsplit=1)
        if len(parts) != 2 or not parts[0].startswith("Container="):
          raise ValueError("Invalid connection string. Expected 'azure+blob+container://Container=<container_name>;<connection_string>'")
        from kv.azure.blob import BlobContainerKV
        from azure.storage.blob.aio import BlobServiceClient
        container = parts[0].split('=')[1]
        client = lambda: BlobServiceClient.from_connection_string
        return BlobContainerKV(conn_str)
    if conn_str.startswith("sqlite://"):
        return SQLKV(conn_str)
    raise ValueError(f"Unknown connection string: {conn_str}")